# Converting the PDF to Excel

### first attempt, simple but too intutitive

In [5]:
import pandas as pd

# read txt
with open('castToTxt.txt', 'r') as file:
    lines = file.readlines()

# using a DF
data = []


# skip head line
for line in lines[1:]:
    parts = line.split(maxsplit=2)  # split into 3 parts: Ser, Category, SubCategory
    if len(parts) == 1:
        # only Ser, no Category or SubCategory
        ser = parts[0]
        category = None
        sub_category = None
    if len(parts) == 2: # SubCategory missing
        ser, category = parts
        sub_category = None
    elif len(parts) == 3:
        ser, category, sub_category = parts
    
    try:
        ser = int(ser)
    except ValueError:
        continue
    
    data.append([ser, category, sub_category])
    # data.append([int(ser), category, sub_category])

# add tp DF
df = pd.DataFrame(data, columns=['Ser', 'Category', 'SubCategory'])

# print df
df.head(15)

,Ser,Category,SubCategory
0,1,Agriculture,None
1,2,Agricultural,Equipment\n
2,3,None,None
3,4,None,None
4,5,None,None
5,6,None,None
6,7,None,None
7,8,None,None
8,9,None,None
9,10,None,None


In [7]:
df.to_excel('output.xlsx', index=False)

print("DataFrame saved to output.xlsx")

DataFrame saved to output.xlsx


    It looks like we have a skeleton, but a few issues exist, lets see if we can get subcategory to work

### Second attempt, Categories & Sub categories only

    we will start out with the first 1/2, spliting categories & subcategories 

In [29]:
import re # allows multiple words

with open('castToTxtFirst.txt', 'r') as file:
    lines = file.readlines()

# list of SER values that should be categories
category_ser_list = [
    1, 152, 341, 535, 749, 927, 1011, 1070, 1316, 1384, 1467, 1511, 1545, 1560, 
    1566, 1645, 1815, 1950, 1994, 2017, 2074, 2344, 2513, 2580, 2633, 3039, 
    3121, 3207, 3388, 3549, 3604, 3680, 3764, 3819, 3872, 4070, 4122, 4260, 
    4316, 4406, 4476
]

# category_ser_list = list(range(1, 4596))  # SER values 1 to 20 as categories

data = []
current_category = None

# # each line
for line in lines[1:]:
    
    line = line.strip()             # keeps empty lines
    if not line:  # check for empty lines
        data.append([None, None, None])
        continue
    
    match = re.match(r'(\d+)\s+(.*)', line)
    if not match:
        continue
    
    ser = int(match.group(1))
    sub_category = match.group(2)
    
    if ser in category_ser_list:
        current_category = sub_category
        data.append([ser, None, current_category])
    else:
        category = current_category
        data.append([ser, category, sub_category])

df = pd.DataFrame(data, columns=['Ser', 'Category', 'SubCategory'])


    This section of code makes sure empty columns for a future subsub category & subsubsub category are included

In [30]:
all_ser_values = list(range(1, 4596))

missing_data = {
    'Ser': [],
    'Category': [],
    'SubCategory': []
}

existing_sers = set(df['Ser'])
for ser in all_ser_values:
    if ser in existing_sers:
        continue  # skip if SER already exists
    
    missing_data['Ser'].append(ser)
    missing_data['Category'].append(None)
    missing_data['SubCategory'].append(None)

# append missing data & sort
missing_df = pd.DataFrame(missing_data)
df = pd.concat([df, missing_df], ignore_index=True)
df = df.sort_values(by='Ser')

    This section fixes the final formatting error (category names being misaligned)

In [31]:
category_ser_list = [
    1, 152, 341, 535, 749, 927, 1011, 1070, 1316, 1384, 1467, 1511, 1545, 1560, 
    1566, 1645, 1815, 1950, 1994, 2017, 2074, 2344, 2513, 2580, 2633, 3039, 
    3121, 3207, 3388, 3549, 3604, 3680, 3764, 3819, 3872, 4070, 4122, 4260, 
    4316, 4406, 4476
]

# update category names in DF
for index, row in df.iterrows():
    if row['Ser'] in category_ser_list:
        category_name = row['SubCategory']
        df.at[index, 'Category'] = category_name
        df.at[index, 'SubCategory'] = ''

    & finally this one commits it to Excel

In [32]:

df.to_excel('output2a.xlsx', index=False)

print("DataFrame saved to output.xlsx")

DataFrame saved to output.xlsx


    We have 1 more job left in this section, this was manually done by using the array of category SER's, we will need a similar array before section 3

In [113]:
filtered_values = pd.DataFrame(df)

for x in filtered_values.index:
    if filtered_values.loc[x, 'Category'] != None:
        filtered_values.drop(x, inplace = True)
filtered_values = filtered_values.drop(['Category', 'SubCategory'], axis=1)
filtered_values.values.flatten()

print(filtered_values)


       Ser
909      3
910      4
911      5
912      6
913      7
...    ...
4590  4591
4591  4592
4592  4593
4593  4594
4594  4595

[3686 rows x 1 columns]


### Third attempt, Sub Sub (Categories & Sub Categories)